In [1]:
import os

import requests
from requests import Response
from tqdm.autonotebook import tqdm

/tmp/ipykernel_6048/340828846.py:5: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
TOKEN = "vk1.a.sQZ83a9pepTHMagHg7o-l_nh1LG9NMyj_VoOqLZq-3vvtlAIsDgprcahHOYcb0oudWxy3_4fy_aKy9xRe3z2NMK1M7DfiwgglkePVY9LMnjOy1Zy5SlNTqWIBx2sGGXYjwzixvwuRPYiDEge6OWsl-M4HxESN38TC1SRavfZSoxeKMw4fMU5Fmcn8DTkYjzS"

In [3]:
class VKApi:
    def __init__(self, token: str, v: str = "5.131"):
        self.TOKEN = token
        self.VERSION = v

    def get(self, url: str, **params) -> Response:
        default_params = {
            "access_token": self.TOKEN,
            "v": self.VERSION,
        }
        return requests.get(f"https://api.vk.com/method/{url}", params=default_params | params)

In [4]:
vk_api = VKApi(TOKEN)

In [5]:
group_domain = "hseperm"

In [6]:
response = vk_api.get("groups.getById", group_id=group_domain)

In [7]:
owner_id = -response.json()["response"][0]["id"]

In [8]:
response = vk_api.get("photos.getAlbums", owner_id=owner_id)

In [9]:
albums = response.json()["response"]["items"]

In [10]:
folder = "saved"
if not os.path.exists(folder):
    os.mkdir(folder)

In [11]:
for album in tqdm(albums):
    photos = []
    for i in range(album["size"] // 1000 + 1):
        album_data = vk_api.get(
            "photos.get",
            album_id=album["id"],
            owner_id=album["owner_id"],
            count=1000,
            offset=i * 1000,
        ).json()["response"]
        for photo in album_data["items"]:
            photos.append(max(photo["sizes"], key=lambda x: x.get("height")))

    path = os.path.join(folder, album["title"])
    if not os.path.exists(path):
        os.mkdir(path)

    for photo in tqdm(photos, desc=album["title"]):
        response = requests.get(photo["url"])
        file_name = photo["url"].split("/")[-1].split("?")[0]
        with open(os.path.join(path, file_name), "wb") as f:
            f.write(response.content)

  0%|          | 0/4 [00:00<?, ?it/s]

Выпускной бакалавриата 2022:   0%|          | 0/30 [00:00<?, ?it/s]

Новое общежитие НИУ ВШЭ – Пермь:   0%|          | 0/13 [00:00<?, ?it/s]

В чем сила твоего дома, Вышка?:   0%|          | 0/8 [00:00<?, ?it/s]

Наши выпускники:   0%|          | 0/10 [00:00<?, ?it/s]

In [45]:
members = vk_api.get(
    "groups.getMembers",
    group_id=group_domain,
    fields=["sex"],  # , "activities", "bdate", "career", "country", "city", "education", "relation", "universities"],
).json()

In [25]:
members = members["response"]["items"]

In [30]:
friend = vk_api.get(
    "friends.get",
    user_id=members[0]["id"],
    count=10000,
).json()

In [34]:
len(user["response"]["items"])

1000

In [36]:
members = members["response"]["items"][0]

In [46]:
members

{'response': {'count': 3229,
  'items': [{'id': 140141,
    'sex': 2,
    'first_name': 'Юрий',
    'last_name': 'Айдаров',
    'can_access_closed': True,
    'is_closed': False},
   {'id': 339124,
    'sex': 2,
    'first_name': 'Roman',
    'last_name': 'Dijalov',
    'can_access_closed': True,
    'is_closed': False},
   {'id': 381263,
    'sex': 1,
    'first_name': 'Елена',
    'last_name': 'Зарубей',
    'can_access_closed': True,
    'is_closed': False},
   {'id': 556605,
    'sex': 1,
    'first_name': 'Aleksandra',
    'last_name': 'Soldatova',
    'can_access_closed': True,
    'is_closed': False},
   {'id': 915663,
    'sex': 2,
    'first_name': 'Дмитрий',
    'last_name': 'Гергерт',
    'can_access_closed': True,
    'is_closed': False},
   {'id': 1013753,
    'sex': 1,
    'first_name': 'Нина',
    'last_name': 'Трошева',
    'can_access_closed': True,
    'is_closed': False},
   {'id': 1252788,
    'sex': 1,
    'first_name': 'Юля',
    'last_name': 'Зорина',
    'can_ac

In [44]:
members["response"]["items"][0]

TypeError: 'Response' object is not subscriptable

In [43]:
members.request.url

'https://api.vk.com/method/groups.getMembers?access_token=vk1.a.sQZ83a9pepTHMagHg7o-l_nh1LG9NMyj_VoOqLZq-3vvtlAIsDgprcahHOYcb0oudWxy3_4fy_aKy9xRe3z2NMK1M7DfiwgglkePVY9LMnjOy1Zy5SlNTqWIBx2sGGXYjwzixvwuRPYiDEge6OWsl-M4HxESN38TC1SRavfZSoxeKMw4fMU5Fmcn8DTkYjzS&v=5.131&group_id=hseperm&fields=sex&fields=activities&fields=bdate&fields=career&fields=country&fields=city&fields=education&fields=relation&fields=universities'